In [1]:
# Load environment variables and set up auto-reload
from dotenv import load_dotenv
load_dotenv()


%load_ext autoreload
%autoreload 2

# Full Docker Monitor

In [2]:
from utils import format_message_content

from langchain.chat_models import init_chat_model

from docker_image_monitor.version_agent import version_agent
from docker_image_monitor.research_agent import research_agent
from langchain_core.messages import HumanMessage


# ===== CONFIGURATION =====

di_list = {"images":[
    {"name":"nginx",
     "current": "1.29.1"},
    {"name":"mariadb",
     "current": "10.6"},
    {"name":"redis",
     "current": "8.2.1"},
]}

# ===== Version Agent Prompt =====

def generate_brief(name:str):
    return f"Are there any updates of the docker image {name} within the last week on the officially Docker Hub website?"


# ===== Research Agent Prompt  =====
def generate_research_brief(new:str):
    return (
        f"{new}. Find and analyze release notes/changelogs for this specific docker image."
        f"Compare it to the previous version. Evaluate the changes in terms of their criticality. Give a clear recommendation for the development team.")



# ===== Full Docker Monitor =====

def monitor(images: list):
    for img in images:
        brief = generate_brief(img["name"])
        result = version_agent.invoke({"researcher_messages": [HumanMessage(content=f"{brief}.")]},{"recursion_limit": 25})
        short_result = format_message_content(result['researcher_messages'][-1])
        print("version_agent result:")
        print(short_result)
        if "update: yes" in short_result:
            print("analyze and evaluate changes...")
            research_brief = generate_research_brief(short_result)
            research_result = research_agent.invoke({"researcher_messages": [HumanMessage(content=f"{research_brief}.")]},{"recursion_limit": 30})
            short_research_result = format_message_content(research_result['researcher_messages'][-1])

        else:
            short_research_result = f"No updates found for {img["name"]}!"
        print(short_research_result)
        print("==============")



In [3]:
monitor(di_list["images"])

version_agent result:
- update: yes
- the name of the docker image: nginx
- the version number of the latest release: 1.29.1

The official nginx Docker image on Docker Hub was updated 3 days ago, which confirms an update within the last week. The latest version is 1.29.1.
analyze and evaluate changes...
Docker Image: nginx  
Version: 1.29.1 (compared to previous stable 1.28.0)

Summary of Changes:
- The primary focus of nginx 1.29.1 is a critical security fix addressing CVE-2025-53859 in the ngx_mail_smtp_module. This vulnerability affects setups where nginx operates as an SMTP proxy server and represents a high risk if left unpatched.
- Protocol improvements include multiple fixes to HTTP/3 (e.g., limiting prefix integer encoding length, header processing logic corrections) and HTTP/2 for enhanced stability and better compliance with protocol standards.
- A new feature was introduced: TLS certificate compression support, which reduces data transfer during TLS handshakes and can improv